In [2]:
%load_ext autoreload
%autoreload 2

import json
import pandas as pd
import grammar as lt
from collections import Counter
from tqdm import tqdm

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


2023-01-28 09:34:18.112073: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-28 09:34:18.951837: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-01-28 09:34:18.951881: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-01-28 09:34:18.951886: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [3]:
df = []
files = !ls data/grammar_*.json
for file in files:
    df.append(pd.read_json(file))
df = pd.concat(df, axis=0, ignore_index=True)

In [8]:
ctr = lt.err_counter(df.to_dict(orient='records'))
ctr.most_common(5)

[('Possible spelling mistake. ‘enrolment’ is British English.', 286283),
 ('Possible spelling mistake found.', 121332),
 ('Possible spelling mistake. ‘enrolments’ is British English.', 91140),
 ('An apostrophe may be missing.', 32906),
 ('This word is normally spelled with a hyphen.', 29850)]

In [9]:
df = df.to_dict(orient='records')
df[0]

{'question_id': 121,
 'matches': [{'message': 'Possible spelling mistake. ‘enrolment’ is British English.',
   'shortMessage': '',
   'replacements': [{'value': 'enrollment',
     'shortDescription': 'English (US)'}],
   'offset': 4,
   'length': 9,
   'context': {'text': 'The enrolment ratio of female students in tertiary sc...',
    'offset': 4,
    'length': 9},
   'sentence': 'The enrolment ratio of female students in tertiary schools in Panama in 2006 is 53.5071105957.',
   'type': {'typeName': 'Other'},
   'rule': {'id': 'MORFOLOGIK_RULE_EN_US',
    'description': 'Possible spelling mistake',
    'issueType': 'misspelling',
    'urls': [{'value': 'https://languagetool.org/insights/post/re-vs-er/#british-english-prefers-doubling-consonants-doesn%E2%80%99t-it'}],
    'category': {'id': 'TYPOS', 'name': 'Possible Typo'}},
   'ignoreForIncompleteSentence': False,
   'contextForSureMatch': 0}]}

In [10]:
for err in df:
    newmatches = []
    for match in err['matches']:
        if 'British English' not in match['message']:
            newmatches.append(match)
    err['matches'] = newmatches
ctr = lt.err_counter(df)
ctr.most_common(5)

[('Possible spelling mistake found.', 121332),
 ('An apostrophe may be missing.', 32906),
 ('This word is normally spelled with a hyphen.', 29850),
 ('A determiner may be missing.', 12971),
 ('With the plural noun ‘sexes’, the verb inflection ‘is’ is not correct.',
  12169)]

In [14]:
hyphens = lt.filter_lt_messages(df, message='hyphen')
repls = []
for err in hyphens:
    for match in err['matches']:
        s = match['sentence']
        if match['message'].endswith('a hyphen.'):
            start = match['offset']
            end = start + match['length']
            repls.append(s[start:end])
set(repls)

{'20-50',
 '6-59',
 'Non renewable',
 'non concessional',
 'non residents',
 'self employed',
 'well developed'}

In [15]:
apos = lt.filter_lt_messages(df, message='apostrophe')
apos[0]

{'question_id': 126816,
 'matches': [{'message': 'An apostrophe may be missing.',
   'shortMessage': 'Possible typo',
   'replacements': [{'value': "males'"},
    {'value': "male's"},
    {'value': "malis's"}],
   'offset': 29,
   'length': 5,
   'context': {'text': 'The percentage of unemployed males population in the United States in 2010...',
    'offset': 29,
    'length': 5},
   'sentence': 'The percentage of unemployed males population in the United States in 2010 is 17.3299999237.',
   'type': {'typeName': 'Other'},
   'rule': {'id': 'POSSESSIVE_APOSTROPHE',
    'subId': '1',
    'sourceFile': 'grammar.xml',
    'description': 'Possessive apostrophe error',
    'issueType': 'typographical',
    'urls': [{'value': 'https://languagetool.org/insights/post/possessive-apostrophe/'}],
    'category': {'id': 'TYPOS', 'name': 'Possible Typo'}},
   'ignoreForIncompleteSentence': True,
   'contextForSureMatch': 13}]}

In [16]:
from spacy import load
nlp = load('en_core_web_md')

In [27]:
def get_phrase(match):
    doc = nlp(match['sentence'])
    start = match['offset']
    end = start + match['length']
    for chunk in doc.noun_chunks:
        if chunk.start_char <= start and chunk.end_char >= end:
            return chunk
    raise ValueError('Phrase not found!')

In [32]:
phrases = []
for err in apos:
    for match in err['matches']:
        if match['message'] == 'An apostrophe may be missing.':
            break
    try:
        # phrase = get_phr
        phrases.append(get_phrase(match))
    except ValueError:
        continue

In [33]:
len(phrases)

25092

In [34]:
len(apos)

25113

In [35]:
set([k.text for k in phrases])

{'The Armed forces personnel',
 'The Benefits incidence',
 'The average unemployed females population',
 'The average unemployed males population',
 'The greatest unemployed females population',
 'The greatest unemployed males population',
 'The highest unemployed females population',
 'The highest unemployed males population',
 'The least unemployed females population',
 'The least unemployed males population',
 'The lowest unemployed females population',
 'The lowest unemployed males population',
 'The maximum unemployed females population',
 'The maximum unemployed males population',
 'The mean unemployed females population',
 'The mean unemployed males population',
 'The median unemployed females population',
 'The median unemployed males population',
 'The minimum unemployed females population',
 'The minimum unemployed males population',
 'The top two unemployed females population',
 'The top two unemployed males population',
 'The unemployed females population',
 'The unemployed

In [37]:
deter = lt.filter_lt_messages(df, message='determiner')

In [40]:
repls = []
for err in deter:
    for match in err['matches']:
        s = match['sentence']
        if match['message'] == 'A determiner may be missing.':
            start = match['offset']
            end = start + match['length']
            repls.append(s[start:end])
set(repls)

{'In Isle of Man',
 'In Turks and Caicos Islands',
 'in Isle of Man',
 'in Turks and Caicos Islands',
 'of Isle of Man',
 'of Turks and Caicos Islands'}

In [41]:
ctr.most_common(5)

[('Possible spelling mistake found.', 121332),
 ('An apostrophe may be missing.', 32906),
 ('This word is normally spelled with a hyphen.', 29850),
 ('A determiner may be missing.', 12971),
 ('With the plural noun ‘sexes’, the verb inflection ‘is’ is not correct.',
  12169)]

In [43]:
infl = lt.filter_lt_messages(df, message='verb inflection')
sents = [k['matches'][0]['sentence'] for k in infl]

In [46]:
import random

In [48]:
random.choices(sents, k=10)

['The difference between the Number of enrolments of both sexes in Mali and the Number of enrolments of both sexes in Jamaica is -1096.0 in the 1988.',
 'In the 1978, the difference between the % of repeaters of the Netherlands and the % of repeaters of Algeria is -6.3255.',
 'The Amount of exports (as % of merchandise exports) of Agricultural raw materials in Kenya is greater than the Amount of exports (as % of merchandise exports) of Fuel in Korea (Republic) by 5.7815.',
 'In 2009, the % of firms in Poland is 14.7.',
 'The percentage of male workers in services in Poland exceeds the percentage of male workers in agriculture in Ukraine by -28.9000015259.',
 'Across Education levels, the highest Number of enrolments of both sexes in Norway is 383499.0.',
 'In 2011, the Number of enrolments of both sexes of Tunisia is 991312.0.',
 'The net concessional disbursements from IMF is more than the average in 0 years.',
 'The sum of the amount spent on the travel items of national tourists in 

In [49]:
doc = nlp('In the graph, the total percentage of exports to different countries in Uruguay is 124.1487804552.')
verb, subject = lt.find_root_nsubj(doc)
verb, subject

(is, percentage)

In [50]:
lt.has_agreement(verb, subject)

(True, is)

In [82]:
nlp = load('en_core_web_sm')

In [89]:
non_agree = []
for doc in nlp.pipe(sents):
    try:
        verb, subject = lt.find_root_nsubj(doc)
        if not lt.has_agreement(verb, subject)[0]:
            non_agree.append(doc)
    except (IndexError, ValueError):
        print(doc)

The average per year cost of damage caused due to CO2 emissions in Mexico is 1386004698.978.
The average per year cost of damage caused due to CO2 emissions in Honduras is 34280472.45504462.
The average per year cost of damage caused due to CO2 emissions in Belize is 1482542.1101903876.
The average per year cost of damage caused due to CO2 emissions in Ethiopia is 27898588.18946522.
The average per year cost of damage caused due to CO2 emissions in Mauritius is 5390193.741583058.
The average per year cost of damage caused due to CO2 emissions in Afghanistan is 43866011.018948935.
The average per year percentage of urban population the Using sanitation services in Nicaragua is 68.64.
The average per year percentage of urban population the Using sanitation services in Lesotho is 35.5.
The average per year percentage of rural population the Using sanitation services in Lebanon is 81.675.
The average per year percentage of rural population the Using sanitation services in Jordan is 97.05.


In [91]:
random.choices(non_agree, k=10)

[In the US$, the Taxes on products in Chad is 21007158.144744.,
 In 2010, the Armed forces personnel in Barbados is 610.0.,
 The net concessional disbursements from IMF is higher than 20000000 the US$ in 0 countries.,
 Across years, the minimum Total subscribers in Dominica is 7300.0.,
 Across years, the maximum service imports in Thailand is 46001742824.,
 In the graph, the total commercial service exports in Botswana is 1311408852.212872.,
 In Brazil, the net financial flows from IBRD is 0.,
 In 2008, the Total subscribers of Colombia is 1772088.0.,
 In 6 years, the net financial flows from IBRD is more than 160000000 the US$.,
 The co2 emissions due to manufacturing industries in Myanmar is not less than that in Cyprus.]

In [93]:
lt.fix_verb_subject_agreement(nlp('In Brazil, the net financial flows from IBRD is 0.'))

'In Brazil, the net financial flows from IBRD are 0.'

In [94]:
len(set([k.text for k in non_agree]))

9000

In [95]:
len(non_agree)

9223